In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torchsummary import summary

%config InlineBackend.figure_formats = ["svg"]

In [2]:
df = pd.read_csv("./Datasets/Graduate Admissions/Admission_Predict_Ver1.1.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          500 non-null    int64  
 2   TOEFL Score        500 non-null    int64  
 3   University Rating  500 non-null    int64  
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 35.3 KB


In [4]:
df.duplicated().sum()

0

In [5]:
df.drop(columns=['Serial No.'],inplace=True)

X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [8]:
X_train_scaled = torch.tensor(X_train_scaled)
X_test_scaled  = torch.tensor(X_test_scaled)

y_train        = torch.tensor(np.reshape(y_train.values, (y_train.shape[-1], 1)))
y_test         = torch.tensor(np.reshape(y_test.values, (y_test.shape[-1], 1)))

In [9]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

(torch.Size([400, 7]),
 torch.Size([100, 7]),
 torch.Size([400, 1]),
 torch.Size([100, 1]))

In [10]:
# build model architecture
ANN_regressor = nn.Sequential(
    nn.Linear(in_features = X_train_scaled.shape[-1], out_features = 7),
    nn.ReLU(),
    nn.Linear(in_features = 7, out_features = 7),
    nn.ReLU(),
    nn.Linear(in_features = 7, out_features = 1)
)

ANN_regressor

Sequential(
  (0): Linear(in_features=7, out_features=7, bias=True)
  (1): ReLU()
  (2): Linear(in_features=7, out_features=7, bias=True)
  (3): ReLU()
  (4): Linear(in_features=7, out_features=1, bias=True)
)

In [11]:
summary(ANN_regressor, X_train_scaled.shape);

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 400, 7]              56
├─ReLU: 1-2                              [-1, 400, 7]              --
├─Linear: 1-3                            [-1, 400, 7]              56
├─ReLU: 1-4                              [-1, 400, 7]              --
├─Linear: 1-5                            [-1, 400, 1]              8
Total params: 120
Trainable params: 120
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 0.00
Estimated Total Size (MB): 0.06


In [12]:
# learning_rate
learning_rate = 0.01

# loss function
lossfun = nn.MSELoss()

# optimizer
optimizer = torch.optim.Adam(ANN_regressor.parameters(), lr = learning_rate)

In [13]:
%%time

# train the model
epochs = 100
losses = torch.zeros(epochs)

for epoch in range(epochs):
    # forward propagation
    y_hat = ANN_regressor(X_train_scaled.float())
    
    # compute loss
    loss          = lossfun(y_hat, y_train.float())
    losses[epoch] = loss
    
    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)